In [1]:
#import thư viện connect my sql
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy.dialects.mysql import insert
from sqlalchemy import Table, MetaData
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import text
from pandasql import sqldf
import pymysql
import pandas as pd
import numpy as np

In [2]:
###create engine
#define
username = 'DT47_UIT'
password = '198198'
host = 'localhost'

DB_NAME = 'Tiki_Product'

# Create a connection object
databaseServerIP            = host
databaseUserName            = username
databaseUserPassword        = password
newDatabaseName             = DB_NAME
charSet                     = "utf8mb4"
cusrorType                  = pymysql.cursors.DictCursor

In [3]:
def create_table(database: str, table: str, datatype: str):
    try:
        username = 'DT47_UIT'
        password = '198198'
        host = 'localhost'
        engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
        with engine.connect() as conn:
            conn.execute(text(f'DROP TABLE IF EXISTS {table};'))
            conn.execute(text(f'CREATE TABLE {table}({datatype})'))
            print('Table is created....')
    except SQLAlchemyError as e:
        print("Error while connecting to MySQL", e)

In [4]:
connectionInstance = pymysql.connect(host=databaseServerIP, 
                                     user=databaseUserName, 
                                     password=databaseUserPassword, 
                                     db=newDatabaseName, 
                                     charset=charSet, 
                                     cursorclass=cusrorType)

In [5]:
# Create a connection to MySQL
database_connection = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{DB_NAME}")

In [6]:
#test connect
try:
    cursor = connectionInstance.cursor()  # tạo một đối tượng con trỏ
    cursor.execute("SELECT DATABASE()")  # thực hiện một truy vấn đơn giản để kiểm tra kết nối
    result = cursor.fetchone()  # lấy kết quả
    print(f"Connected to database: {result['DATABASE()']}")
except pymysql.OperationalError as e:
    print(f"Error connecting to the database: {e}")


Connected to database: tiki_product


In [7]:
def create_table(database: str, table: str, datatype: str):
    username = 'DT47_UIT'
    password = '198198'
    host = 'localhost'
    engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
    with engine.connect() as conn:
        # xuat danh sach bang trong mysql
        result = conn.execute(text(f"SHOW TABLES LIKE '{table}'"))
        if result.fetchone():
            print(f'Table {table} already exists.')    #neu co thi bo qua
        else:
            try:
                conn.execute(text(f'CREATE TABLE {table}({datatype})'))     #khong co thi create
                print('Table is created....')
            except SQLAlchemyError as e:
                print("Error while connecting to MySQL", e)


**ETL**

Extract

In [91]:
# Extract data from the two tables
temporary_table = pd.read_sql('SELECT * FROM temporary_table', con=database_connection)
product_id = pd.read_sql('SELECT * FROM product_id', con=database_connection)
product_id.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36285 entries, 0 to 36284
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     36285 non-null  int64 
 1   id        36285 non-null  object
 2   category  36285 non-null  object
 3   sort      36285 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.1+ MB


In [92]:
# Convert the 'id' column of the 'product_id' DataFrame to int64
product_id['id'] = product_id['id'].astype('int64')
temporary_table['breadcrumb_category_id'] = temporary_table['breadcrumb_category_id'].astype('object')
temporary_table['master_id'] = temporary_table['master_id'].astype('object')
temporary_table['category_id'] = temporary_table['category_id'].fillna(0).astype(int)
# merge the two DataFrames
merged_data = pd.merge(temporary_table, product_id, on='id', how='left')
merged_data.rename(columns={'id': 'product_id'}, inplace=True) #rename after merged

merged_data

,product_id,master_id,sku,all_time_quantity_sold,breadcrumb_name,breadcrumb_category_id,category_id,quantity_sold,rating_average,review_count,...,product_name,brand_id,brand_name,categories_name,description_short,day_ago_created,fulfillment_type,index,category,sort
0,54645,54645,5004548693125,855.0,Thiết Bị Âm Thanh và Phụ Kiện,1815,1804,855.0,4.5,83,...,Tai nghe chụp tai Logitech H150 - 2 jack 3.5mm...,18984,Logitech,Tai Nghe Có Dây,"ÂM THANH TỐTVới âm thanh nổi hoàn toàn, tai ng...",3820,None,158171,thiet-bi-kts-phu-kien-so,top_seller
1,54665,54665,5006348510336,5783.0,Phụ kiện máy tính và Laptop,1815,1815,5783.0,4.7,696,...,Chuột không dây Logitech M185 - Hãng chính hãng,18984,Logitech,Thiết Bị Số - Phụ Kiện Số,Tận hưởng thế giới không dâyChuột không dây Lo...,3820,None,149991,thiet-bi-kts-phu-kien-so,newest
2,122012,122012,6804968803567,137.0,Phụ kiện máy tính và Laptop,1815,1815,137.0,4.7,36,...,Combo chuột phím không dây Logitech MK240 - Hà...,18984,Logitech,Thiết Bị Số - Phụ Kiện Số,Combo tiện lợiBộ Bàn Phím Và Chuột Không Dây M...,3126,None,150201,thiet-bi-kts-phu-kien-so,newest
3,125781,125781,6508902455033,10198.0,Thiết Bị Âm Thanh và Phụ Kiện,1815,1804,10198.0,4.5,1381,...,Tai Nghe Nhét Tai Sony Fontopia MDR-E9LP - Hàn...,18641,Sony,Tai Nghe Có Dây,Thiết kế gọn nhẹTai Nghe Sony Fontopia MDR-E9L...,3084,None,148981,thiet-bi-kts-phu-kien-so,newest
4,126000,126000,6505679403830,5144.0,Thiết Bị Âm Thanh và Phụ Kiện,1815,1804,5144.0,4.6,638,...,Tai Nghe Chụp Tai Sony MDR-ZX110AP - Hàng Chín...,18641,Sony,Tai Nghe Có Dây,Thiết kế gọn nhẹTai Nghe Sony MDR-ZX110AP Chụp...,3082,None,149412,thiet-bi-kts-phu-kien-so,newest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36071,263582266,263582266,5193017296340,0.0,Đèn & thiết bị chiếu sáng,1883,23244,NaN,0.0,0,...,Đui Đèn Xoáy Ốp Trần / Ốp Tường VINAKIP 60W 25...,111461,OEM,Linh kiện đèn,Đui Đèn Xoáy Ốp Trần / Ốp Tường VINAKIP 60W 25...,0,None,240480,nha-cua-doi-song,newest
36072,263582282,263582282,9117217266142,0.0,Làm Đẹp - Sức Khỏe,1520,8224,NaN,0.0,0,...,NẾN THƠM TINH DẦU SẢ CHANH [LY 50G - HỘP 2 LY ...,689369,R2D,Nến thơm,MÔ TẢ SẢN PHẨM: -Chúng ta đều cảm thấy t...,0,None,230478,lam-dep-suc-khoe,"price,desc"
36073,263582336,263582336,8434175796660,0.0,Nhà Cửa - Đời Sống,1883,4381,NaN,0.0,0,...,Ghế họp Juno Sofa NT T'he one GL406 580 x 630 ...,7426417,Tundo,Bàn ghế làm việc,"Mô tả sản phẩm ghế GL406 khung thép mạ, đệm tự...",0,None,240479,nha-cua-doi-song,newest
36074,263582341,263582341,7690341190697,0.0,Nhà Cửa - Đời Sống,1883,2948,NaN,0.0,0,...,Kìm - Cờ lê cầm tay 10 inch - Kìm dùng để giữ ...,623837,TRENY,"Cờ lê, mỏ lết, búa các loại",2023/06/2KÌM CẦM TAY ĐA NĂNG – KÌM SỬA CHỮA CƠ...,0,None,240478,nha-cua-doi-song,newest


In [63]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36076 entries, 0 to 36075
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      36076 non-null  int64  
 1   master_id               36076 non-null  object 
 2   sku                     36076 non-null  object 
 3   all_time_quantity_sold  36076 non-null  float64
 4   breadcrumb_name         36076 non-null  object 
 5   breadcrumb_category_id  36076 non-null  object 
 6   category_id             36076 non-null  int32  
 7   quantity_sold           15287 non-null  float64
 8   rating_average          36076 non-null  float64
 9   review_count            36076 non-null  int64  
 10  original_price          36076 non-null  int64  
 11  price                   36076 non-null  int64  
 12  discount                36076 non-null  int64  
 13  discount_rate           36076 non-null  int64  
 14  list_price              36076 non-null

In [127]:
merged_data.rename(columns={'index': 'idx'}, inplace=True)#thay doi cau truc bang de connect
cols = merged_data.columns.tolist()
cols.insert(0, cols.pop(cols.index('idx')))
merged_data = merged_data.reindex(columns=cols)
merged_data

,idx,product_id,master_id,sku,all_time_quantity_sold,breadcrumb_name,breadcrumb_category_id,category_id,quantity_sold,rating_average,...,is_visible,product_name,brand_id,brand_name,categories_name,description_short,day_ago_created,fulfillment_type,category,sort
0,158171,54645,54645,5004548693125,855.0,Thiết Bị Âm Thanh và Phụ Kiện,1815,1804,855.0,4.5,...,None,Tai nghe chụp tai Logitech H150 - 2 jack 3.5mm...,18984,Logitech,Tai Nghe Có Dây,"ÂM THANH TỐTVới âm thanh nổi hoàn toàn, tai ng...",3820,None,thiet-bi-kts-phu-kien-so,top_seller
1,149991,54665,54665,5006348510336,5783.0,Phụ kiện máy tính và Laptop,1815,1815,5783.0,4.7,...,None,Chuột không dây Logitech M185 - Hãng chính hãng,18984,Logitech,Thiết Bị Số - Phụ Kiện Số,Tận hưởng thế giới không dâyChuột không dây Lo...,3820,None,thiet-bi-kts-phu-kien-so,newest
2,150201,122012,122012,6804968803567,137.0,Phụ kiện máy tính và Laptop,1815,1815,137.0,4.7,...,None,Combo chuột phím không dây Logitech MK240 - Hà...,18984,Logitech,Thiết Bị Số - Phụ Kiện Số,Combo tiện lợiBộ Bàn Phím Và Chuột Không Dây M...,3126,None,thiet-bi-kts-phu-kien-so,newest
3,148981,125781,125781,6508902455033,10198.0,Thiết Bị Âm Thanh và Phụ Kiện,1815,1804,10198.0,4.5,...,None,Tai Nghe Nhét Tai Sony Fontopia MDR-E9LP - Hàn...,18641,Sony,Tai Nghe Có Dây,Thiết kế gọn nhẹTai Nghe Sony Fontopia MDR-E9L...,3084,None,thiet-bi-kts-phu-kien-so,newest
4,149412,126000,126000,6505679403830,5144.0,Thiết Bị Âm Thanh và Phụ Kiện,1815,1804,5144.0,4.6,...,None,Tai Nghe Chụp Tai Sony MDR-ZX110AP - Hàng Chín...,18641,Sony,Tai Nghe Có Dây,Thiết kế gọn nhẹTai Nghe Sony MDR-ZX110AP Chụp...,3082,None,thiet-bi-kts-phu-kien-so,newest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36071,240480,263582266,263582266,5193017296340,0.0,Đèn & thiết bị chiếu sáng,1883,23244,Null,0.0,...,None,Đui Đèn Xoáy Ốp Trần / Ốp Tường VINAKIP 60W 25...,111461,OEM,Linh kiện đèn,Đui Đèn Xoáy Ốp Trần / Ốp Tường VINAKIP 60W 25...,0,None,nha-cua-doi-song,newest
36072,230478,263582282,263582282,9117217266142,0.0,Làm Đẹp - Sức Khỏe,1520,8224,Null,0.0,...,None,NẾN THƠM TINH DẦU SẢ CHANH [LY 50G - HỘP 2 LY ...,689369,R2D,Nến thơm,MÔ TẢ SẢN PHẨM: -Chúng ta đều cảm thấy t...,0,None,lam-dep-suc-khoe,"price,desc"
36073,240479,263582336,263582336,8434175796660,0.0,Nhà Cửa - Đời Sống,1883,4381,Null,0.0,...,None,Ghế họp Juno Sofa NT T'he one GL406 580 x 630 ...,7426417,Tundo,Bàn ghế làm việc,"Mô tả sản phẩm ghế GL406 khung thép mạ, đệm tự...",0,None,nha-cua-doi-song,newest
36074,240478,263582341,263582341,7690341190697,0.0,Nhà Cửa - Đời Sống,1883,2948,Null,0.0,...,None,Kìm - Cờ lê cầm tay 10 inch - Kìm dùng để giữ ...,623837,TRENY,"Cờ lê, mỏ lết, búa các loại",2023/06/2KÌM CẦM TAY ĐA NĂNG – KÌM SỬA CHỮA CƠ...,0,None,nha-cua-doi-song,newest


#define create_table

In [103]:
def create_table(database: str, table: str, datatype: str):
    username = 'DT47_UIT'
    password = '198198'
    host = 'localhost'
    engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
    with engine.connect() as conn:
        # xuat danh sach bang trong mysql
        result = conn.execute(text(f"SHOW TABLES LIKE '{table}'"))
        if result.fetchone():
            print(f'Table {table} already exists.')    #neu co thi bo qua
        else:
            try:
                conn.execute(text(f'CREATE TABLE {table}({datatype})'))     #khong co thi create
                print('Table is created....')
            except SQLAlchemyError as e:
                print("Error while connecting to MySQL", e)


In [102]:
def insert_ignore(df, table_name, engine):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(df.columns)
    placeholders = ', '.join(['%s'] * len(df.columns))
    query  = f"INSERT IGNORE INTO {table_name} ({cols}) VALUES ({placeholders})"
    with engine.connect() as conn:
        try:
            cursor = conn.connection.cursor()
            cursor.executemany(query, tuples)
            conn.connection.commit()
        except Exception as e:
            conn.connection.rollback()
            raise e

In [98]:
table = "merged_final"
datatype = '''
            idx INT,
            product_id INT PRIMARY KEY,
            master_id VARCHAR(255),
            sku VARCHAR(255),
            all_time_quantity_sold FLOAT,
            breadcrumb_name VARCHAR(255),
            breadcrumb_category_id VARCHAR(255),
            category_id int,
            quantity_sold FLOAT,
            rating_average FLOAT,
            review_count INT,
            original_price INT,
            price INT,
            discount INT,
            discount_rate INT,
            list_price INT,
            price_usd VARCHAR(255),
            order_count VARCHAR(255),
            inventory_status VARCHAR(255),
            is_visible VARCHAR(255),
            product_name VARCHAR(255),
            brand_id INT,
            brand_name VARCHAR(255),
            categories_name VARCHAR(255),
            description_short VARCHAR(255),
            day_ago_created INT,
            fulfillment_type VARCHAR(255),
            category VARCHAR(255),
            sort VARCHAR(255)'''
create_table("Tiki_Product", table=table, datatype=datatype)

Table merged_final already exists.


In [45]:
table = "Dim_Brand"
datatype = '''
            Brand_id INT PRIMARY KEY,
            Brand_name VARCHAR(255)'''
create_table("Tiki_Product",table=table, datatype=datatype)


Table is created....


In [77]:
table = "Dim_Category"
datatype = '''
            category_id int PRIMARY KEY,
            breadcrumb_category_id VARCHAR(255),
            breadcrumb_name VARCHAR(255)'''
create_table("Tiki_Product",table=table, datatype=datatype)


Table is created....


In [124]:
table = "Dim_Product"
datatype = '''
            Product_id INT PRIMARY KEY,
            Sku VARCHAR(255),
            Product_name VARCHAR(255),
            Description_short VARCHAR(255),
            Day_ago_created FLOAT,
            Fulfillment_type VARCHAR(255),
            Inventory_status VARCHAR(255)'''
create_table("Tiki_Product",table=table, datatype=datatype)


Table is created....


In [135]:
table = "Fact_Sales"
datatype = '''
            Sales_id INT PRIMARY KEY,
            All_time_quantity_sold FLOAT,
            Quantity_sold FLOAT,
            Original_price FLOAT,
            Price FLOAT,
            Discount FLOAT,
            Discount_rate FLOAT,
            List_price FLOAT,
            Price_usd FLOAT,
            Order_count INT,
            Product_id INT,
            Brand_id INT,
            Category_id INT'''
create_table("Tiki_Product", table=table, datatype=datatype)

Table is created....


In [104]:
connectionInstance = pymysql.connect(host=databaseServerIP, 
                                     user=databaseUserName, 
                                     password=databaseUserPassword, 
                                     db=newDatabaseName, 
                                     charset=charSet, 
                                     cursorclass=cusrorType)
cursor = connectionInstance.cursor()
cursor.execute("SELECT * FROM Tiki_Product.Dim_Brand;")
result = cursor.fetchone()
print(result)


{'Brand_id': 17825, 'Brand_name': 'HP'}


In [109]:
merged_data = merged_data.drop_duplicates()#check truoc khi save
merged_data.fillna(value='Null', inplace=True)

table_name_brand = 'merged_final'
insert_ignore(merged_data, table_name_brand, database_connection)
merged_data

,idx,product_id,master_id,sku,all_time_quantity_sold,breadcrumb_name,breadcrumb_category_id,category_id,quantity_sold,rating_average,...,is_visible,product_name,brand_id,brand_name,categories_name,description_short,day_ago_created,fulfillment_type,category,sort
0,158171,54645,54645,5004548693125,855.0,Thiết Bị Âm Thanh và Phụ Kiện,1815,1804,855.0,4.5,...,None,Tai nghe chụp tai Logitech H150 - 2 jack 3.5mm...,18984,Logitech,Tai Nghe Có Dây,"ÂM THANH TỐTVới âm thanh nổi hoàn toàn, tai ng...",3820,None,thiet-bi-kts-phu-kien-so,top_seller
1,149991,54665,54665,5006348510336,5783.0,Phụ kiện máy tính và Laptop,1815,1815,5783.0,4.7,...,None,Chuột không dây Logitech M185 - Hãng chính hãng,18984,Logitech,Thiết Bị Số - Phụ Kiện Số,Tận hưởng thế giới không dâyChuột không dây Lo...,3820,None,thiet-bi-kts-phu-kien-so,newest
2,150201,122012,122012,6804968803567,137.0,Phụ kiện máy tính và Laptop,1815,1815,137.0,4.7,...,None,Combo chuột phím không dây Logitech MK240 - Hà...,18984,Logitech,Thiết Bị Số - Phụ Kiện Số,Combo tiện lợiBộ Bàn Phím Và Chuột Không Dây M...,3126,None,thiet-bi-kts-phu-kien-so,newest
3,148981,125781,125781,6508902455033,10198.0,Thiết Bị Âm Thanh và Phụ Kiện,1815,1804,10198.0,4.5,...,None,Tai Nghe Nhét Tai Sony Fontopia MDR-E9LP - Hàn...,18641,Sony,Tai Nghe Có Dây,Thiết kế gọn nhẹTai Nghe Sony Fontopia MDR-E9L...,3084,None,thiet-bi-kts-phu-kien-so,newest
4,149412,126000,126000,6505679403830,5144.0,Thiết Bị Âm Thanh và Phụ Kiện,1815,1804,5144.0,4.6,...,None,Tai Nghe Chụp Tai Sony MDR-ZX110AP - Hàng Chín...,18641,Sony,Tai Nghe Có Dây,Thiết kế gọn nhẹTai Nghe Sony MDR-ZX110AP Chụp...,3082,None,thiet-bi-kts-phu-kien-so,newest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36071,240480,263582266,263582266,5193017296340,0.0,Đèn & thiết bị chiếu sáng,1883,23244,Null,0.0,...,None,Đui Đèn Xoáy Ốp Trần / Ốp Tường VINAKIP 60W 25...,111461,OEM,Linh kiện đèn,Đui Đèn Xoáy Ốp Trần / Ốp Tường VINAKIP 60W 25...,0,None,nha-cua-doi-song,newest
36072,230478,263582282,263582282,9117217266142,0.0,Làm Đẹp - Sức Khỏe,1520,8224,Null,0.0,...,None,NẾN THƠM TINH DẦU SẢ CHANH [LY 50G - HỘP 2 LY ...,689369,R2D,Nến thơm,MÔ TẢ SẢN PHẨM: -Chúng ta đều cảm thấy t...,0,None,lam-dep-suc-khoe,"price,desc"
36073,240479,263582336,263582336,8434175796660,0.0,Nhà Cửa - Đời Sống,1883,4381,Null,0.0,...,None,Ghế họp Juno Sofa NT T'he one GL406 580 x 630 ...,7426417,Tundo,Bàn ghế làm việc,"Mô tả sản phẩm ghế GL406 khung thép mạ, đệm tự...",0,None,nha-cua-doi-song,newest
36074,240478,263582341,263582341,7690341190697,0.0,Nhà Cửa - Đời Sống,1883,2948,Null,0.0,...,None,Kìm - Cờ lê cầm tay 10 inch - Kìm dùng để giữ ...,623837,TRENY,"Cờ lê, mỏ lết, búa các loại",2023/06/2KÌM CẦM TAY ĐA NĂNG – KÌM SỬA CHỮA CƠ...,0,None,nha-cua-doi-song,newest


load table merged_final

In [111]:
merged_final = pd.read_sql('SELECT * FROM merged_final', con=database_connection)
merged_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36076 entries, 0 to 36075
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   idx                     36076 non-null  int64  
 1   product_id              36076 non-null  int64  
 2   master_id               36076 non-null  object 
 3   sku                     36076 non-null  object 
 4   all_time_quantity_sold  36076 non-null  float64
 5   breadcrumb_name         36076 non-null  object 
 6   breadcrumb_category_id  36076 non-null  object 
 7   category_id             36076 non-null  int64  
 8   quantity_sold           36076 non-null  float64
 9   rating_average          36076 non-null  float64
 10  review_count            36076 non-null  int64  
 11  original_price          36076 non-null  int64  
 12  price                   36076 non-null  int64  
 13  discount                36076 non-null  int64  
 14  discount_rate           36076 non-null

In [112]:
df_brand = merged_final[['brand_id', 'brand_name']].drop_duplicates()
df_brand.fillna(value='Null', inplace=True)
table_name_brand = 'Dim_Brand'
insert_ignore(df_brand, table_name_brand, database_connection)
df_brand

,brand_id,brand_name
0,18984,Logitech
3,18641,Sony
11,50629,ANKER
13,74674,Bigen
17,18991,Cooler Master
...,...,...
35898,48497,Eco
36010,5166975,Anh Lam Store
36022,708725,iMat
36069,467599,behome


In [113]:
df_Category=merged_final[['category_id','breadcrumb_category_id', 'breadcrumb_name']].drop_duplicates()
df_Category['category_id'] = df_Category['category_id'].fillna(0).astype(int)
table_name_brand = 'Dim_Category'
insert_ignore(df_Category, table_name_brand, database_connection)
df_Category

,category_id,breadcrumb_category_id,breadcrumb_name
0,1804,1815,Thiết Bị Âm Thanh và Phụ Kiện
1,1815,1815,Phụ kiện máy tính và Laptop
9,5264,1815,Phụ kiện máy tính và Laptop
11,1815,1815,Phụ Kiện Điện Thoại và Máy Tính Bảng
13,1520,1520,Chăm sóc tóc và da đầu
...,...,...,...
35134,23288,1883,Đồ dùng phòng ăn
35344,8229,1520,Làm Đẹp - Sức Khỏe
35598,28582,1815,Thiết Bị Số - Phụ Kiện Số
35960,23154,1883,Dụng cụ nhà bếp


In [128]:
#product
df_product=merged_final[['product_id','sku', 'product_name','description_short','day_ago_created','fulfillment_type','inventory_status']].drop_duplicates()
table_name_brand = 'Dim_Product'
insert_ignore(df_product, table_name_brand, database_connection)
df_product

,product_id,sku,product_name,description_short,day_ago_created,fulfillment_type,inventory_status
0,54645,5004548693125,Tai nghe chụp tai Logitech H150 - 2 jack 3.5mm...,"ÂM THANH TỐTVới âm thanh nổi hoàn toàn, tai ng...",3820,None,available
1,54665,5006348510336,Chuột không dây Logitech M185 - Hãng chính hãng,Tận hưởng thế giới không dâyChuột không dây Lo...,3820,None,available
2,122012,6804968803567,Combo chuột phím không dây Logitech MK240 - Hà...,Combo tiện lợiBộ Bàn Phím Và Chuột Không Dây M...,3126,None,available
3,125781,6508902455033,Tai Nghe Nhét Tai Sony Fontopia MDR-E9LP - Hàn...,Thiết kế gọn nhẹTai Nghe Sony Fontopia MDR-E9L...,3084,None,available
4,126000,6505679403830,Tai Nghe Chụp Tai Sony MDR-ZX110AP - Hàng Chín...,Thiết kế gọn nhẹTai Nghe Sony MDR-ZX110AP Chụp...,3082,None,available
...,...,...,...,...,...,...,...
36071,263582266,5193017296340,Đui Đèn Xoáy Ốp Trần / Ốp Tường VINAKIP 60W 25...,Đui Đèn Xoáy Ốp Trần / Ốp Tường VINAKIP 60W 25...,0,None,available
36072,263582282,9117217266142,NẾN THƠM TINH DẦU SẢ CHANH [LY 50G - HỘP 2 LY ...,MÔ TẢ SẢN PHẨM: -Chúng ta đều cảm thấy t...,0,None,available
36073,263582336,8434175796660,Ghế họp Juno Sofa NT T'he one GL406 580 x 630 ...,"Mô tả sản phẩm ghế GL406 khung thép mạ, đệm tự...",0,None,available
36074,263582341,7690341190697,Kìm - Cờ lê cầm tay 10 inch - Kìm dùng để giữ ...,2023/06/2KÌM CẦM TAY ĐA NĂNG – KÌM SỬA CHỮA CƠ...,0,None,available


In [132]:
Fact_Sales = merged_final[['product_id', 'all_time_quantity_sold', 'quantity_sold', 'original_price', 'price', 'discount', 'discount_rate', 'list_price', 'price_usd', 'order_count', 'brand_id', 'category_id']]
Fact_Sales = Fact_Sales.rename(columns={'product_id': 'sale_id'})
Fact_Sales

,sale_id,all_time_quantity_sold,quantity_sold,original_price,price,discount,discount_rate,list_price,price_usd,order_count,brand_id,category_id
0,54645,855.0,855.0,328900,267000,61900,19,328900,Null,Null,18984,1804
1,54665,5783.0,5783.0,261000,217000,44000,17,261000,Null,Null,18984,1815
2,122012,137.0,137.0,399000,399000,0,0,399000,Null,Null,18984,1815
3,125781,10198.0,10198.0,199000,116000,83000,42,199000,Null,Null,18641,1804
4,126000,5144.0,5144.0,590000,445000,145000,25,590000,Null,Null,18641,1804
...,...,...,...,...,...,...,...,...,...,...,...,...
36071,263582266,0.0,0.0,8000,8000,0,0,8000,Null,Null,111461,23244
36072,263582282,0.0,0.0,60800,60800,0,0,60800,Null,Null,689369,8224
36073,263582336,0.0,0.0,2022000,2022000,0,0,2022000,Null,Null,7426417,4381
36074,263582341,0.0,0.0,79000,79000,0,0,79000,Null,Null,623837,2948


In [136]:
#sales
Fact_Sales = merged_final[['product_id', 'all_time_quantity_sold', 'quantity_sold', 'original_price', 'price', 'discount', 'discount_rate', 'list_price', 'price_usd', 'order_count', 'brand_id', 'category_id']]
Fact_Sales = Fact_Sales.rename(columns={'product_id': 'sales_id'})

table_name_brand = 'Fact_Sales'
insert_ignore(Fact_Sales, table_name_brand, database_connection)
Fact_Sales

,sales_id,all_time_quantity_sold,quantity_sold,original_price,price,discount,discount_rate,list_price,price_usd,order_count,brand_id,category_id
0,54645,855.0,855.0,328900,267000,61900,19,328900,Null,Null,18984,1804
1,54665,5783.0,5783.0,261000,217000,44000,17,261000,Null,Null,18984,1815
2,122012,137.0,137.0,399000,399000,0,0,399000,Null,Null,18984,1815
3,125781,10198.0,10198.0,199000,116000,83000,42,199000,Null,Null,18641,1804
4,126000,5144.0,5144.0,590000,445000,145000,25,590000,Null,Null,18641,1804
...,...,...,...,...,...,...,...,...,...,...,...,...
36071,263582266,0.0,0.0,8000,8000,0,0,8000,Null,Null,111461,23244
36072,263582282,0.0,0.0,60800,60800,0,0,60800,Null,Null,689369,8224
36073,263582336,0.0,0.0,2022000,2022000,0,0,2022000,Null,Null,7426417,4381
36074,263582341,0.0,0.0,79000,79000,0,0,79000,Null,Null,623837,2948
